In [1]:
import os 
import pandas as pd
from pydub import AudioSegment
import re

In [2]:
guitar_path = '../IDMT-SMT-CHORDS/guitar'
non_guitar_path = '../IDMT-SMT-CHORDS/non_guitar'
raw_data_path = '../data/raw'    

In [3]:
guitar_annotations = pd.read_csv(os.path.join(guitar_path,'guitar_annotation.lab'), sep = '\t', header = 0, names =['start_time', 'end_time', 'chord'])
non_guitar_annotations = pd.read_csv(os.path.join(non_guitar_path,'non_guitar_annotation.lab'), sep = '\t', header = 0, names =['start_time', 'end_time', 'chord'])

In [4]:
print(f'El numero de acordes que vamos a tener en total es {(273*6)+(576*10)} por lo que vamos a necesitar 4 digitos')

El numero de acordes que vamos a tener en total es 7398 por lo que vamos a necesitar 4 digitos


In [8]:
def obtain_type_of_sound(file_name):
    pattern = r'_([A-Z]\w*)\.wav$' # r'_(\w+)\.wav$'
    match = re.search(pattern, file_name)   
    if match:
        return match.group(1)
    
def obtain_root_note(chord):     
    fundamental_note, chord_type = re.split(':', chord, maxsplit=1)
    return fundamental_note, chord_type

def preprocess_audio(file_name, index, label_dict, label_dataframe, guitar_flag):

    audio = AudioSegment.from_wav(file_name)
    duration = len(audio) 

    start_time = 0 
    end_time = 2000
     
    dataframe_index = 0 
    while start_time < duration:    
        # Save the audio fragment
        index_str = f"{str(index).zfill(4)}.wav"
        trimmed_audio = audio[start_time:end_time]
        trimmed_audio.export('../data/trimmed_audio/' + index_str, format="wav")
        start_time = end_time
        end_time = end_time + 2000
        
        # Get label to the dictionary and update it 
        dataframe = guitar_annotations if guitar_flag else non_guitar_annotations   

        
        label_dict[index_str] = dataframe.iloc[dataframe_index]['chord']  

        type_of_sound = obtain_type_of_sound(file_name) 
        fundamental_note, chord_type = obtain_root_note(label_dict[index_str])
        instrument = 'guitar' if guitar_flag else 'non_guitar'  
        label_dataframe.loc[index] = [index_str, label_dict[index_str], instrument, type_of_sound, fundamental_note, chord_type]  
        
        # Update indexes    
        dataframe_index += 1     
        index += 1

    return index, label_dict, label_dataframe

In [ ]:
label_dict = {} 
label_dataframe = pd.DataFrame(columns=['file_name', 'label', 'instrument', 'type_of_sound', 'root_note', 'chord_type'])        
directory_path = os.makedirs('trimmed_audio', exist_ok=True)
for path, _, filenames in os.walk(raw_data_path):
        index = 0
        for filename in filenames:
                if filename.endswith('.wav'):
                        guitar_flag = 'guitar' in filename 
                        index, label_dict, label_dataframe = preprocess_audio(os.path.join(path, filename), index, label_dict, label_dataframe, guitar_flag)

dictionary = pd.DataFrame.from_dict(label_dict, orient='index')  
dictionary.to_csv('label_dictionary.csv', header=False)              
label_dataframe.to_csv('label_dataframe.csv', header=True, index=False)